In [ ]:
import math, itertools, random
import altair as alt
import misfunciones

# Lab 6-c: Encontrando cajas DnaA

En este último lab de este bloque vamos a centrarnos en encontrar las cajas DnaA en E. Coli. 
¿Cómo? Si recuerdas, en la última tarea te encargué que creases una función `kmeros_frecuentes(secuencia, k)` que devolviese un mapa de frecuencias de cada k-mero de la longitud que se pasase como parámetro.

Además, te pedí que probases tu código con el origen de replicación del vibrio cholerae y los parámetros k=3, 4, 5, 6, 7, 8, 9. Vamos a hacerlo. 

In [ ]:
def lee_genoma(ruta_fichero):
  fd = open(ruta_fichero)
  genome = fd.read()
  fd.close()
  return genome

In [ ]:
tp_genoma = lee_genoma('../data/Thermotoga-petrophila.txt')
ec_genoma = lee_genoma('../data/E-coli.txt')
vc_genoma = lee_genoma('../data/vibrio_cholerae.txt')

fd = open("./oric.fasta")
lines  = fd.readlines()
vc_oric = lines[1].strip('\n') # origen de replicación del Vibrio Cholerae.
tp_oric = lines[3].strip('\n') # origen de replicación de la Thermotoga Petrophila.
fd.close()

In [ ]:
def kmeros_frecuentes(secuencia, k):
    freq = {}
    n = len(secuencia)
    for i in range(n-k+1):
        kmero = secuencia[i:i+k]
        if kmero in freq:
            freq[kmero] += 1
        else:
            freq[kmero] = 1
    return freq

In [ ]:
kmeros_frecuentes(vc_oric, 2)

In [ ]:
kmeros_frecuentes(vc_oric, 3)

In [ ]:
# nos saltamos unos pocos...
kmeros_frecuentes(vc_oric, 9)

Si te fijas en el mapa de frecuencias, existen 3 k-meros que aparecen 3 veces: `atgatcaag`, `cttgatcat`, `tcttgatca` y `ctcttgatc`. 

¿Cómo de sorprendente es que un k-mero de longitud 9 aparezca en una cadena de longitud 500? Pues bastante, aproximadamente 1/1300!

Para aproximarlo, y al tratarse de una cadena relativamente bastante pequeña, podemos usar la fórmula general:

$\frac{N-t(k-1) \choose t}{A^{t \cdot k}}$


Vamos a verlo mejor. Si queremos ver qué k-meros salen más veces, es cuestión simplemente de filtrar el mapa de frecuencias, así:

In [ ]:
def top_kmeros(secuencia, k):
    kmeros = []
    freqs = kmeros_frecuentes(secuencia, k)
    m = max(freqs.values())
    for key in freqs:
        if freqs[key] == m:
            kmeros.append(key)
        # add each key to words whose corresponding frequency value is equal to m
    return kmeros, m

In [ ]:
top_kmeros(vc_oric, 9)

Espera un momento. Entre estos cuatro k-meros hay dos `'atgatcaag'` y `'cttgatcat'` que son inversos complementarios del otro! Esto sí que es sorprendente! Además, biológicamente tiene sentido ya que la proteína DnaA puede adherirse a cualquier de los dos sentidos para iniciar la replicación. Probablemente estas sean las cajas DnaA del V. Cholerae...

Pero antes, para asegurarnos, vamos a buscar dónde aparecen en todo el genoma (puede ser que aparezca recurrentemente). 

In [ ]:
def busca_kmero(secuencia, kmero):
    posiciones = [] # output variable
    for i in range(len(secuencia) - len(kmero) + 1):
        if secuencia[i:i+len(kmero)] == kmero:
            posiciones.append(i)
    return posiciones

In [ ]:
busca_kmero(vc_genoma, 'atgatcaag'.upper())

Pues resulta que no, que aunque aparece en más posiciones, este k-mero sólo se "acumula" alrededor de la posición 152000, justo en el origen de replicación. Más evidencia para apoyar que es una caja DnaA. 

En fin, vamos a aplicar el mismo procedimiento a la zona que pensamos que es el origen de replicación del E.Coli, que vimos que era 3923620, de acuerdo al diagrama de sesgo. Buscaremos en una ventana de 500 bp a partir de estas posición, ya que sabemos que es un tamaño habitual en estos microorganismos:

In [ ]:
kmeros_frecuentes(ec_genoma[3923620:3923620+500], 9)

In [ ]:
top_kmeros(ec_genoma[3923620:3923620+500], 9)

Vaya, no hay kmeros que aparezcan 3 veces! Vamos a volver a mirar el origen de replicación del V. Cholerae, a ver si obtenemos alguna pista...

Si estamos atent@s, nos daremos cuenta de que hay otros k-meros que son sorprendentemente parecidos a los que habíamos encontrado como posibles cajas DnaA, estos son 

`ATGATCAAC` <-> `ATGATCAAG` (cambia el último carácter)
`CATGATCAT,` <-> `CTTGATCAT` (cambia el segundo carácter).

¿Y si pasase algo parecido en el E. Coli? Vamos a comprobarlo.

Para modificar nuestro método de contar patrones, primero tendremos que permitir hacer comparaciones "difusas" basadas en el número de carácteres diferentes entre dos cadenas. Si este número está por debajo de un límite dado, las cadenas serán consideradas iguales :)

Implementa una función `distancia(kmero_1, kmero_2)` que devuelva el número de caracteres diferentes que hay entre dos k-meros dados. Haciendo uso de esta función, genera otra llamada `busca_kmero_aprox(secuencia, kmero, d)` que devuelva cuántas veces aparece un k-mero (y otros k-meros parecidos con distancia <= d) en una secuencia dada (considera también la inversa complementaria del k-mero).


In [ ]:
def distancia(kmero_1, kmero_2):
    #Tu código aquí
        

In [ ]:
def cuenta_kmero_aprox(secuencia, kmero, d):
    #Tu código aquí
    

In [ ]:
assert(cuenta_kmero_aprox("AACAAGCATAAACATTAAAGAG", "AAAAA", 2) == 13) # asegurate de que tú código pasa este test

Perfecto! ¿Te has fijado? `AAAAA` no aparece en la secuencia pero aún así es el k-mero más frecuente! Ten esto en cuenta en el este ejercicio: Vamos a redefinir la función `kmeros_frecuentes` para que funcione usando la cuenta aproximada que acabas de hacer. 

Como hemos visto, en esta nueva modalidad de búsqueda difusa, un k-mero puede no estar presente en la cadena y, a pesar de ello, ser el más frecuente. Por ello, tendremos que probar con todos los $4^{k}$ k-meros posibles. 

Vamos a implementarlo en código:

In [ ]:
def kmeros_frecuentes_aprox(secuencia, k, d):
    #Tu código aquí

In [ ]:
%time resultados  = kmeros_frecuentes_aprox(ec_genoma[3923620:3923620+500], 9, 1) # tarda mucho! Dale tiempo

In [ ]:
len(resultados)

In [ ]:
valor_max = max(resultados.values())

In [ ]:
top_kmeros = []
for k,v in resultados.items():
    if v == valor_max:
        top_kmeros.append(k)

print(top_kmeros, valor_max)
        

¿Qué ocurre aquí? Parece que hay bastantes k-meros que aparecen 4 veces en la región que hemos identificado como el origen de replicación...¿Cuál de ellos podría ser la caja DnaA?

Vamos a intentar agrupar los resultados considerando que:
1. La caja puede aparecer como una cadena o su inversa complementaria.
2. La caja puede contener mutaciones de, como mucho en este caso, 1 elemento. Por lo tanto, a todos los efectos, las cadenas que difieran en 1 caracter de una cadena cadena (o de su inversa complementaria) irán al mismo grupo que dicha cadena.

In [ ]:
def agrupa_resultados(top_kmeros):
    grupos = []
    while len(top_kmeros) > 0:
        kmero = top_kmeros.pop()
        grupo_temp = [kmero]
        for i in range(len(top_kmeros)):
            if distancia(kmero, top_kmeros[i]) <= 1 or kmero == misfunciones.inv_comp(top_kmeros[i]) or distancia(misfunciones.inv_comp(kmero), top_kmeros[i]) <= 1:
                grupo_temp.append(top_kmeros[i])

        top_kmeros = [kmero for kmero in top_kmeros if kmero not in grupo_temp]
        grupos.append(sorted(grupo_temp)) # ordenamos cada grupo lexicográficamente para ver sus diferencias.
    
    return sorted(grupos, key=lambda kmeros: kmeros[0]) # mantenemos el orden lexicográfico en la lista de grupos

In [ ]:
agrupa_resultados(top_kmeros)

Enhorabuena, has encontrado la caja DnaA en E.Coli!!!! 